In [ ]:
# Import libraries
import cv2
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from tqdm import tqdm
from tifffile import imread
import csv

from stardist import random_label_cmap, _draw_polygons, fill_label_holes, relabel_image_stardist, calculate_extents, gputools_available
from stardist.models import Config2D, StarDist2D, StarDistData2D
from stardist.plot import render_label, render_label_pred
from stardist.matching import matching, matching_dataset
from skimage.morphology import binary_dilation

from csbdeep.utils import Path, normalize

%matplotlib inline

## Initializing StarDist 2D Model for Segmentation

In this part of the script, we set up the environment for image segmentation using the StarDist 2D model. The process involves setting a random seed for reproducibility, initializing a label color map for visualization, and loading a pre-trained StarDist 2D model.
### Steps:

   - Set Random Seed:
        A fixed random seed is set to ensure consistency in operations that involve randomness.

   - Initialize Label Color Map:
        A random label color map is created for use in visualizing segmentation results.

   - Load Pre-Trained StarDist 2D Model:
        The StarDist 2D model is instantiated and loaded from a specified directory, ready for segmentation tasks.

In [ ]:
np.random.seed(42)
lbl_cmap = random_label_cmap()

In [ ]:
# Set a fixed random seed for reproducibility
np.random.seed(42)

# Generate a random label color map for visualization
lbl_cmap = random_label_cmap()

# Load the pre-trained StarDist 2D model
model = StarDist2D(None, name='model name', basedir='path/to/your/model/directory')


## Loading and Preprocessing an External Image for Segmentation

This segment of the code is dedicated to loading an external image, converting it to grayscale, and inspecting its shape and data type. This is a preparatory step for image segmentation tasks, particularly when using models like StarDist 2D.

### Steps:

1. **Load External Image:**
   - The image is loaded from a specified file path using OpenCV's `cv2.imread` function.

2. **Convert to Grayscale:**
   - The loaded image is converted to grayscale format, which is a common requirement for many image processing tasks and models.

3. **Inspect Image Properties:**
   - The shape and data type of the image are inspected. This information is crucial for understanding the image's dimensions and ensuring compatibility with segmentation models.

In [ ]:
# Load the image from the specified file path
my_img = cv2.imread("path/to/your/image/Image.TIFF")

# Convert the image to grayscale
my_img = cv2.cvtColor(my_img, cv2.COLOR_BGR2GRAY)

# Print the shape and data type of the image
print("Image shape:", my_img.shape)
print("Image data type:", my_img.dtype)

## Segmenting a Normalized Image Using StarDist 2D Model

This part of the script focuses on normalizing an external image and then applying the pre-trained StarDist 2D model to perform image segmentation. The normalization step is crucial for standardizing the pixel values, while the StarDist model predicts instances (segmentation) in the image.

### Steps:

1. **Normalization:**
   - The image is normalized using the `normalize` function. Normalization scales the pixel values to a specific percentile range, which is essential for consistent processing.
   - The `axis_norm` parameter determines the axes along which the normalization is performed.

2. **Segmentation with StarDist 2D:**
   - The pre-trained StarDist 2D model is used to segment the normalized image.
   - The `predict_instances` method performs the segmentation, returning both the segmented image and additional details.
   - `verbose=True` enables detailed logging during the prediction process.
   - `nms_thresh=0.1` sets the non-maximum suppression threshold, which affects the detection sensitivity.

In [ ]:
# Define the axis for normalization
axis_norm = (0, 1)

# Normalize the image
my_img_norm = normalize(my_img, 1, 99.8, axis=axis_norm)

# Perform segmentation using the StarDist 2D model
segmented_img, details_img = model.predict_instances(my_img_norm, verbose=True, nms_thresh=0.1)

## Visualizing the Original and Segmented Images

In [ ]:
# Configure the overall size of the figure
plt.rcParams["figure.figsize"] = (50, 50)

# Plot the original (normalized) image
plt.subplot(1, 2, 1)
plt.imshow(my_img_norm, cmap="gray")
plt.axis("off")
plt.title("Input Image")

# Plot the segmented image
plt.subplot(1, 2, 2)
# Overlay segmentation labels on the original image
plt.imshow(render_label(segmented_img, img=my_img_norm, cmap="RdYlGn", alpha=0.5))
plt.axis("off")
plt.title("Prediction")

## Counting Predicted Objects in Segmented Image

This code snippet focuses on using the StarDist 2D model to predict segmentation labels for an image and then count the number of segmented objects within it. This is a valuable step in quantifying the results of the segmentation process.

### Steps:

1. **Predict Segmentation Labels:**
   - The `predict_instances` method of the StarDist 2D model is used to predict the segmentation labels for the normalized image.
   
2. **Count Objects:**
   - The unique labels are identified, and their count is determined using `np.unique`. Each unique label represents a different object.
   - The background label is excluded from the count (hence the subtraction of 1), ensuring that only actual objects are counted.

In [ ]:
# Predict the segmentation labels for the normalized image
labels, _ = model.predict_instances(my_img_norm)

# Count the number of unique objects, excluding the background
num_objects = len(np.unique(labels)) - 1  # Subtract 1 to exclude the background label

# Print the number of predicted objects
print(f"Number of predicted objects: {num_objects}")

## Rendering Segmentation Results on a Black Background

This code snippet is designed to overlay the segmentation results from the StarDist model onto a black background. This approach highlights the segmented areas, making them stand out visually.

### Process:

- **Create a Black Background Image:**
  - A black image (`black_img`) is created with the same dimensions as the normalized input image. This serves as the backdrop for the segmentation results.

- **Overlay Segmentation Labels:**
  - The `render_label` function overlays the segmented labels onto the black background. The labels are displayed using a specific colormap (`RdYlGn`) with a chosen level of transparency (`alpha=0.7`).

- **Visualization:**
  - The resulting image is displayed in a plot with a figure size of 6x6 inches. Axes are turned off to focus solely on the segmentation results.

In [ ]:
# Render the segmentation results on a black background
black_img = np.zeros_like(my_img_norm)  # Create a black image with the same shape as the normalized image
plt.figure(figsize=(6, 6))
plt.imshow(render_label(segmented_img, img=black_img, cmap="RdYlGn", alpha=0.7))  # Overlay the segmented labels on the black image
plt.axis('off')  # Hide the axes for a cleaner presentation

In [ ]:
# Expand detected regions to excapsulate the crystals completely

## Expanding Segmentation Mask and Rendering on Image

This code snippet involves expanding the segmentation mask obtained from the StarDist model and then rendering it onto the original image for visual examination. 

1. **Expand Segmentation Mask:**
   - Using `binary_dilation` to expand the segmented mask. This enlarges the segmented areas for better visibility.
   - A footprint (structuring element) with a shape of `(30, 30)` defines how the dilation is applied.

2. **Rendering Expanded Mask on Original Image:**
   - The `render_label` function overlays the expanded mask onto the original image.
   - The 'RdYlGn' colormap is used with an alpha value of 0.7, providing a clear and distinct visualization of the expanded mask.

In [ ]:
# Perform binary dilation on the segmented mask to expand the segmented areas
expanded_mask = binary_dilation(segmented_img, footprint=np.ones((30, 30)))

# Set up the figure for visualization
plt.figure(figsize=(10, 8))

# Overlay the expanded mask on the original image and display
plt.imshow(render_label(expanded_mask, img=my_img_norm, cmap="RdYlGn", alpha=0.7))

In [ ]:
# To keep the region from the original image that matched the expanded predicted region and render the rest 
# of the image as white, I used the expanded mask as a boolean index to select the pixels in the original 
# image that belong to the predicted region, and set the remaining pixels to white

## Cropping Expanded Regions from Segmented Image

This code snippet demonstrates the process of isolating the expanded regions from a segmented image. The expanded mask is used to selectively keep the regions of interest from the original image while setting the remaining areas to white. This approach highlights the segmented areas, effectively cropping them out from the rest of the image.

### Steps:

1. **Create a Copy of the Original Image:**
   - The original image is copied to retain its original state while allowing modifications on the copy.

2. **Apply Expanded Mask to Image:**
   - The expanded mask is used as a boolean index to select pixels in the original image that belong to the predicted region.
   - Pixels not in the predicted region are set to white, diminishing their visual impact.

3. **Visualize the Cropped Image:**
   - A new figure is created for displaying the results.
   - The `render_label` function overlays the expanded mask on a black background image, clearly marking the segmented areas.
   - The cropped image is then displayed, showing the segmented areas in their original form against a white background.


In [ ]:
# Cropping the expanded regions from the segmented image
# and rendering the rest of the image as white

# Copy the original normalized image
my_img_cropped = my_img_norm.copy()

# Using the expanded mask as a boolean index, keep the pixels in the original image that belong to the predicted region.
# Set the remaining pixels to white (value of 1.0)
my_img_cropped[~expanded_mask] = 1.0

# Setting up the figure for visualization
plt.figure(figsize=(10,8))
black_img = np.zeros_like(my_img_norm)

# Overlay the expanded mask on the black image for clear visualization of the segmented areas
render_label(expanded_mask, img=black_img, cmap="RdYlGn", alpha=0.7)

# Display the cropped image where the segmented regions are highlighted and the rest of the image is dimmed
plt.imshow(my_img_cropped, cmap="gray", alpha=0.7)

## Creating a Binary Mask from the Expanded Segmented Regions

This code snippet focuses on creating a binary mask from the expanded segmented regions of an image. The process involves modifying a copy of the normalized image to highlight the segmented areas and then converting it into a binary image using a specified threshold.

### Process:

1. **Copy and Modify the Original Image:**
   - A copy of the normalized image is created.
   - The expanded mask is applied to this copy. Pixels in the segmented regions are kept as they are, while the others are set to black (value of 0), and then subsequently to white (value of 1).

2. **Threshold Application:**
   - A threshold value is defined to distinguish the foreground from the background in the image.
   - The image is converted into a binary format (black and white) based on this threshold, resulting in a binary mask.

3. **Visualizing the Binary Mask:**
   - The binary mask is visualized in grayscale to clearly show the segmented regions against the background.

In [ ]:
# Create a copy of the normalized image
masked_img = np.copy(my_img_norm)

# Apply the expanded mask to the image copy, setting non-segmented regions to black and then to white
masked_img[~expanded_mask] = 0  # Set non-segmented regions to 0 (black)
masked_img[~expanded_mask] = 1  # Set non-segmented regions to 1 (white)

# Define a threshold value to differentiate the foreground from the background
threshold_value = 0.5

# Convert the masked image to a binary image based on the threshold
binary_img = (masked_img > threshold_value).astype(np.uint8)

# Set up the figure for visualization
plt.figure(figsize=(10, 8))

# Display the binary image to visualize the mask
plt.imshow(binary_img, cmap="gray")

## Analyzing Contours and Calculating Circularity

This code snippet involves finding contours in a binary image, calculating the circularity and size of each contour, and labeling them. The aim is to analyze the shapes of segmented regions and quantify their circularity.

### Steps:

1. **Find Contours:**
   - Using OpenCV's `cv2.findContours` method, contours within the binary image are detected. These contours represent the edges of the segmented regions.

2. **Calculate Circularity and Size:**
   - For each contour, its area and perimeter are calculated. Using these, the circularity is determined, which is a measure of how close the shape is to a perfect circle.

3. **Filter Contours Based on Area:**
   - Only contours within a specified area range are considered to filter out too small or too large regions.

4. **Draw and Label Contours:**
   - Each valid contour is drawn and labeled with a number on a separate image for visualization.

5. **Create Data Table:**
   - A table is constructed to store the circularity and size data for each contour. This data can be further analyzed or saved.

6. **Save Data as CSV (Optional):**
   - The data table is converted into a pandas DataFrame and can be saved as a CSV file for external analysis.

In [ ]:
# Find the contours in the binary image
contours, hierarchy = cv2.findContours(binary_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Initialize an image to draw contours
img_contours = np.zeros_like(binary_img)
img_contours.fill(255)  # Fill with white background

# Initialize a list to store data for each contour
table_data = []

# Loop through each contour to calculate circularity and size
for i, contour in enumerate(contours):
    area = cv2.contourArea(contour)
    # Filter out contours that are too small or too large
    if area < 50 or area > 3000:
        continue
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        continue
    circularity = 4 * np.pi * area / (perimeter ** 2)

    # Draw filled contours on the image
    cv2.drawContours(img_contours, contours, i, (0, 0, 0), thickness=1, lineType=cv2.LINE_AA)
    cv2.drawContours(img_contours, contours, i, (0, 0, 0), thickness=-1, lineType=cv2.LINE_AA)

    # Label each contour with a number
    cv2.putText(img_contours, str(i), (contour[0][0][0]+20, contour[0][0][1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), thickness=2)
    
    # Append contour data to the table
    table_data.append({'Contour': i, 'Circularity': circularity, 'Size': area})

# Convert the table data into a pandas DataFrame
df = pd.DataFrame(table_data)
# Optional: Save the data to a CSV file
# df.to_csv('circularities.csv', index=False)

## Displaying Segmentation Analysis Results

This code snippet is aimed at presenting the results of the segmentation analysis. It involves displaying the total count of detected cells (contours) and the details of each cell, including its circularity and size.

### Steps:

1. **Count Detected Cells:**
   - The total number of cells detected in the segmentation analysis is determined by counting the entries in the DataFrame (`df`).

2. **Print the Total Count:**
   - The total count of detected cells is printed out for quick reference.

3. **Display Detailed Analysis:**
   - The DataFrame containing the analysis details for each cell, such as circularity and size, is printed.
   - Optionally, only the top entries of the DataFrame can be displayed for a concise summary.

In [ ]:
# Display the number of cells (contours) detected
print("Number of cells detected:", len(df))

In [ ]:
# Print the entire DataFrame to show the analysis results for each cell
print(df.to_string(index=False))

# Optional: Print only the top 10 entries of the DataFrame for a concise summary
# print(df.head(10).to_string(index=False))

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img_contours, cmap="gray")